## **Note!**

Make sure you install pygame in your terminal: \
pip install pygame

## **Prototype 7 - Concept**

In [ ]:
import pygame
import sys
import random
import heapq
from datetime import datetime, timedelta
from collections import deque

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)
GREY = (200, 200, 200)
LIGHT_BROWN = (205, 133, 63) # Peru - light brown background

# Constants
WIDTH, HEIGHT = 800, 600
CELL_SIZE = 30 # Increased cell size
COLS = 50 # Reduced number of columns
ROWS = HEIGHT // CELL_SIZE
AGENT_SPEED = 250 # Slightly increased speed
MIN_DISTANCE = COLS // 4
ITEM_TYPES = {
    'speed_up': {'color': ORANGE, 'effect': 1.5, 'duration': 2, 'image_key': 'accelerator'},
    'speed_down': {'color': RED, 'effect': 0.67, 'duration': 2, 'image_key': 'shackle'},
    'reveal_path': {'color': (128, 0, 128), 'effect': True, 'duration': 2, 'image_key': 'god_eye'}
}
ITEM_SPACING = 3
NUM_ITEMS = 25
ITEM_SCALE = 0.7 # Slightly increased item scale for larger cells

# Camera control
camera_x = 0
CAMERA_LERP = 0.1
CAMERA_OFFSET = WIDTH // 4
MAX_CAMERA_X = (COLS * CELL_SIZE) - WIDTH # Adjusted for new COLS

# --- Initialize Pygame ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mega Maze Pathfinder with Images")
clock = pygame.time.Clock()

# --- Load Images ---
try:
    thief_img = pygame.image.load('Thief.PNG').convert_alpha()
    wall_img = pygame.image.load('Wall.PNG').convert_alpha()
    altar_img = pygame.image.load('Altar.PNG').convert_alpha()
    blue_portal_img = pygame.image.load('BluePortal.PNG').convert_alpha()
    accelerator_img = pygame.image.load('Accelerator.PNG').convert_alpha()
    shackle_img = pygame.image.load('Shackle.PNG').convert_alpha()
    god_eye_img = pygame.image.load('GodEye.PNG').convert_alpha()
except pygame.error as e:
    print(f"Error loading one or more images: {e}")
    print("Please ensure Thief.PNG, Wall.PNG, Accelerator.PNG, Shackle.PNG, GodEye.PNG, Altar.PNG, BluePortal.PNG are in the same folder as the script.")
    pygame.quit()
    sys.exit()

# --- Scale Images ---
thief_img = pygame.transform.scale(thief_img, (CELL_SIZE, CELL_SIZE))
wall_img = pygame.transform.scale(wall_img, (CELL_SIZE, CELL_SIZE))
altar_img = pygame.transform.scale(altar_img, (CELL_SIZE, CELL_SIZE))
blue_portal_img = pygame.transform.scale(blue_portal_img, (CELL_SIZE, CELL_SIZE))

item_img_width = int(CELL_SIZE * ITEM_SCALE)
item_img_height = int(CELL_SIZE * ITEM_SCALE)
accelerator_img = pygame.transform.scale(accelerator_img, (item_img_width, item_img_height))
shackle_img = pygame.transform.scale(shackle_img, (item_img_width, item_img_height))
god_eye_img = pygame.transform.scale(god_eye_img, (item_img_width, item_img_height))

ITEM_IMAGES = {
    'accelerator': accelerator_img,
    'shackle': shackle_img,
    'god_eye': god_eye_img
}

class Item:
    def __init__(self, pos, item_type_key): # pos is (row, col)
        self.grid_pos = pos
        # screen_pos is the center of the cell: (center_x, center_y)
        self.screen_pos = (pos[1] * CELL_SIZE + CELL_SIZE // 2,  # col for x
                           pos[0] * CELL_SIZE + CELL_SIZE // 2)  # row for y
        self.type = item_type_key
        self.active = True

    @staticmethod
    def check_item_collision(agent_pos, items_list): # agent_pos is pygame.Vector2
        agent_rect_size = CELL_SIZE // 2
        agent_rect = pygame.Rect(
            agent_pos.x - agent_rect_size // 2,
            agent_pos.y - agent_rect_size // 2,
            agent_rect_size,
            agent_rect_size
        )

        for item_obj in items_list:
            if not item_obj.active:
                continue

            item_collision_width = CELL_SIZE * ITEM_SCALE
            # item_obj.screen_pos is (center_x, center_y)
            item_rect = pygame.Rect(
                item_obj.screen_pos[0] - item_collision_width // 2,  # center_x - width/2
                item_obj.screen_pos[1] - item_collision_width // 2,  # center_y - height/2
                item_collision_width,
                item_collision_width
            )

            if agent_rect.colliderect(item_rect):
                item_obj.active = False
                return item_obj
        return None

class SmoothAgent:
    def __init__(self, start_pos): # start_pos is (row, col)
        self.target_pos = start_pos # Current grid target (row, col)
        self.current_pos = pygame.Vector2(
            start_pos[1] * CELL_SIZE + CELL_SIZE // 2, # col for x
            start_pos[0] * CELL_SIZE + CELL_SIZE // 2  # row for y
        )
        self.base_speed = AGENT_SPEED
        self.speed = self.base_speed
        self.active_effects = deque(maxlen=3)

    def update(self, path, dt):
        # Convert self.target_pos (grid coordinates) to target pixel coordinates
        target_pixel_x = self.target_pos[1] * CELL_SIZE + CELL_SIZE / 2
        target_pixel_y = self.target_pos[0] * CELL_SIZE + CELL_SIZE / 2
        target_pixel_vec = pygame.Vector2(target_pixel_x, target_pixel_y)

        if not path or self.current_pos == target_pixel_vec:
            return

        direction = target_pixel_vec - self.current_pos
        if direction.length() > 0:
            move_distance = self.speed * dt
            if direction.length() <= move_distance:
                self.current_pos = target_pixel_vec
            else:
                direction.scale_to_length(move_distance)
                self.current_pos += direction
            if (self.current_pos - target_pixel_vec).length() < 1: # Snap if very close
                self.current_pos = target_pixel_vec

def bfs_reachable(maze, start_node): # start_node is (row, col)
    visited = set()
    queue = deque([start_node])
    visited.add(start_node)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    while queue:
        current_r, current_c = queue.popleft() # current is (row, col)
        for dr, dc in directions:
            nr, nc = current_r + dr, current_c + dc
            if 0 <= nr < ROWS and 0 <= nc < COLS:
                if maze[nr][nc] == 0 and (nr, nc) not in visited:
                    visited.add((nr, nc))
                    queue.append((nr, nc))
    return visited

def get_valid_path_cells(maze, start_node, end_node):
    from_start = bfs_reachable(maze, start_node)
    from_end = bfs_reachable(maze, end_node)
    return from_start.intersection(from_end)

def generate_items(maze, start_node, end_node):
    valid_cells_on_path = get_valid_path_cells(maze, start_node, end_node)
    valid_positions = [cell for cell in valid_cells_on_path if cell != start_node and cell != end_node]
    generated_items = []
    random.shuffle(valid_positions)
    for pos in valid_positions: # pos is (row, col)
        if len(generated_items) >= NUM_ITEMS:
            break
        too_close = False
        for existing_item in generated_items:
            # existing_item.grid_pos is (row, col)
            if abs(pos[0] - existing_item.grid_pos[0]) < ITEM_SPACING or \
               abs(pos[1] - existing_item.grid_pos[1]) < ITEM_SPACING:
                too_close = True
                break
        if not too_close:
            item_type_key = random.choice(list(ITEM_TYPES.keys()))
            generated_items.append(Item(pos, item_type_key))
    return generated_items

def generate_maze(rows, cols):
    while True:
        maze = [[1 for _ in range(cols)] for _ in range(rows)]
        # start and end are (row, col)
        start_node = (random.randint(0, rows - 1), random.randint(0, cols // 8))
        end_node = (random.randint(0, rows - 1), random.randint(cols - cols // 8, cols - 1))

        # Access tuple elements directly using index
        if abs(start_node[1] - end_node[1]) < MIN_DISTANCE: # Compare columns (index 1)
            continue
        stack = [start_node]
        maze[start_node[0]][start_node[1]] = 0 # Mark start as path
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        while stack:
            current_r, current_c = stack[-1]
            neighbors = []
            for dr, dc in directions:
                nr, nc = current_r + dr * 2, current_c + dc * 2
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] == 1:
                    neighbors.append(((nr, nc), (current_r + dr, current_c + dc)))
            if neighbors:
                (next_r, next_c), (wall_r, wall_c) = random.choice(neighbors)
                maze[wall_r][wall_c] = 0
                maze[next_r][next_c] = 0
                stack.append((next_r, next_c))
            else:
                stack.pop()
        for _ in range(rows * cols // 20): # Make maze less perfect
            r, c = random.randint(0, rows - 1), random.randint(0, cols - 1)
            if maze[r][c] == 1:
                path_neighbors = 0
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] == 0:
                        path_neighbors +=1
                if path_neighbors > 0 :
                    maze[r][c] = 0
        if astar(maze, start_node, end_node):
            return maze, start_node, end_node

def heuristic(a, b): # a and b are (row, col) tuples
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(maze, pos): # pos is (row, col)
    neighbors = []
    r, c = pos # Unpack position tuple
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < ROWS and 0 <= nc < COLS and maze[nr][nc] == 0:
            neighbors.append((nr, nc))
    return neighbors

def reconstruct_path(came_from, current_node):
    path = [current_node]
    while current_node in came_from:
        current_node = came_from[current_node] # Access dictionary directly
        path.append(current_node)
    return path[::-1]

def astar(maze, start_node, end_node): # start_node and end_node are (row, col)
    open_set = []
    heapq.heappush(open_set, (0, start_node))
    came_from = {}
    g_score = {(r, c): float('inf') for r in range(ROWS) for c in range(COLS)}
    g_score[start_node] = 0 # Direct dictionary access
    f_score = {(r, c): float('inf') for r in range(ROWS) for c in range(COLS)}
    f_score[start_node] = heuristic(start_node, end_node) # Direct dictionary access
    open_set_hash = {start_node}

    while open_set:
        _, current_node = heapq.heappop(open_set)
        if current_node in open_set_hash:
            open_set_hash.remove(current_node)
        else:
            continue
        if current_node == end_node:
            return reconstruct_path(came_from, current_node)
        for neighbor in get_neighbors(maze, current_node):
            tentative_g_score = g_score[current_node] + 1 # Direct dictionary access
            if tentative_g_score < g_score[neighbor]: # Direct dictionary access
                came_from[neighbor] = current_node # Direct dictionary access
                g_score[neighbor] = tentative_g_score # Direct dictionary access
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, end_node) # Direct dictionary access
                if neighbor not in open_set_hash:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor)) # Direct dictionary access
                    open_set_hash.add(neighbor)
    return None

def draw_text(surface, text, pos, size=30, color=BLACK, font_name=None):
    font = pygame.font.SysFont(font_name, size)
    text_surface = font.render(text, True, color)
    surface.blit(text_surface, pos)

def update_camera(agent_current_pos_x, dt):
    global camera_x
    target_cam_x = agent_current_pos_x - CAMERA_OFFSET
    target_cam_x = max(0, min(target_cam_x, MAX_CAMERA_X))
    camera_x += (target_cam_x - camera_x) * CAMERA_LERP * dt * 60

# --- Game Initialization ---
maze, start, end = generate_maze(ROWS, COLS) # start and end are (row, col)
original_maze = [row[:] for row in maze]
path = astar(maze, start, end)
items = generate_items(maze, start, end)
agent = SmoothAgent(start)

game_over = False
show_path_temporarily = False

last_block_time = datetime.now()
block_cooldown = 2
path_index = 0

# --- Main Game Loop ---
running = True
while running:
    dt = clock.tick(60) / 1000.0
    time_since_last_block = (datetime.now() - last_block_time).total_seconds()
    can_place_block = time_since_last_block >= block_cooldown

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            if can_place_block:
                mouse_screen_x, mouse_screen_y = pygame.mouse.get_pos()
                world_x = mouse_screen_x + camera_x
                col = int(world_x // CELL_SIZE)
                row = int(mouse_screen_y // CELL_SIZE)
                if 0 <= row < ROWS and 0 <= col < COLS:
                    current_agent_grid_pos = (
                        int(agent.current_pos.y // CELL_SIZE),
                        int(agent.current_pos.x // CELL_SIZE)
                    )
                    action_taken = False
                    if event.button == 1:
                        if maze[row][col] == 0 and (row, col) != start and \
                           (row, col) != end and (row,col) != current_agent_grid_pos :
                            maze[row][col] = 1
                            action_taken = True
                    elif event.button == 3:
                        if maze[row][col] == 1:
                            maze[row][col] = 0
                            action_taken = True
                    if action_taken:
                        last_block_time = datetime.now()
                        new_path = astar(maze, current_agent_grid_pos, end)
                        if new_path:
                            path = new_path
                            path_index = 0
                            if path: agent.target_pos = path[0] # Direct list access
                        else:
                            game_over = True
                            path = []
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                maze = [row[:] for row in original_maze]
                path = astar(maze, start, end)
                items = generate_items(maze, start, end)
                agent = SmoothAgent(start)
                path_index = 0
                if path: agent.target_pos = path[0] # Direct list access
                game_over = False; camera_x = 0; show_path_temporarily = False
                agent.active_effects.clear(); agent.speed = agent.base_speed
            elif event.key == pygame.K_t:
                maze, start, end = generate_maze(ROWS, COLS)
                original_maze = [row[:] for row in maze]
                path = astar(maze, start, end)
                items = generate_items(maze, start, end)
                agent = SmoothAgent(start)
                path_index = 0
                if path: agent.target_pos = path[0] # Direct list access
                game_over = False; camera_x = 0; show_path_temporarily = False
                agent.active_effects.clear(); agent.speed = agent.base_speed

    if not game_over:
        if path and path_index < len(path):
            agent.target_pos = path[path_index] # Direct list access
            target_pixel_pos = pygame.Vector2(
                agent.target_pos[1] * CELL_SIZE + CELL_SIZE // 2, # col for x
                agent.target_pos[0] * CELL_SIZE + CELL_SIZE // 2  # row for y
            )
            if agent.current_pos.distance_to(target_pixel_pos) < max(1.0, agent.speed * dt * 0.5):
                agent.current_pos = target_pixel_pos
                path_index += 1
                if path_index < len(path):
                    agent.target_pos = path[path_index] # Direct list access
                elif agent.target_pos == end :
                    print("Agent reached the end!")
                    pass
        agent.update(path, dt)

        current_time = datetime.now()
        agent.speed = agent.base_speed
        new_speed_modifier = 1.0
        path_reveal_active_this_frame = False

        effects_to_process = list(agent.active_effects)
        for effect in effects_to_process: # effect is a dictionary
            if (current_time - effect['start_time']).total_seconds() > effect['duration']:
                try:
                    agent.active_effects.remove(effect)
                except ValueError:
                    print(f"Warning: Tried to remove an effect that was not found in deque: {effect}")
            else:
                item_type_details = ITEM_TYPES[effect['item_type_key']] # Direct dict access
                if isinstance(item_type_details['effect'], (int, float)):
                     new_speed_modifier *= item_type_details['effect']
                elif item_type_details['effect'] is True and effect['item_type_key'] == 'reveal_path': # Direct dict access
                    path_reveal_active_this_frame = True

        agent.speed = agent.base_speed * new_speed_modifier
        show_path_temporarily = path_reveal_active_this_frame

        collided_item = Item.check_item_collision(agent.current_pos, items)
        if collided_item:
            item_info = ITEM_TYPES[collided_item.type] # Direct dict access
            is_new_effect = True
            for existing_effect in agent.active_effects: # existing_effect is a dictionary
                if existing_effect['item_type_key'] == collided_item.type: # Direct dict access
                    existing_effect['start_time'] = current_time # Direct dict access
                    is_new_effect = False
                    break
            if is_new_effect:
                agent.active_effects.append({
                    'item_type_key': collided_item.type,
                    'start_time': current_time,
                    'duration': item_info['duration']
                })

    update_camera(agent.current_pos.x, dt)

    screen.fill(LIGHT_BROWN) # Set background color
    start_col_on_screen = int(camera_x // CELL_SIZE)
    end_col_on_screen = start_col_on_screen + (WIDTH // CELL_SIZE) + 2
    for r in range(ROWS):
        for c in range(max(0, start_col_on_screen), min(COLS, end_col_on_screen)):
            if maze[r][c] == 1: # Direct list access
                screen.blit(wall_img, (c * CELL_SIZE - camera_x, r * CELL_SIZE))

    # start and end are (row, col)
    screen.blit(altar_img, (start[1] * CELL_SIZE - camera_x, start[0] * CELL_SIZE)) # col for x, row for y
    screen.blit(blue_portal_img, (end[1] * CELL_SIZE - camera_x, end[0] * CELL_SIZE)) # col for x, row for y

    for item_obj in items:
        if item_obj.active:
            item_image_key = ITEM_TYPES[item_obj.type]['image_key'] # Direct dict access
            img_to_draw = ITEM_IMAGES[item_image_key] # Direct dict access
            # item_obj.screen_pos is (center_x, center_y)
            item_draw_x = item_obj.screen_pos[0] - camera_x - img_to_draw.get_width() // 2
            item_draw_y = item_obj.screen_pos[1] - img_to_draw.get_height() // 2 # No y-camera adjustment for items
            screen.blit(img_to_draw, (item_draw_x, item_draw_y))

    if show_path_temporarily and path and not game_over and path_index < len(path):
        for i in range(path_index, len(path)):
            p_r, p_c = path[i] # Direct list access
            pygame.draw.circle(screen, ORANGE,
                               (p_c * CELL_SIZE - camera_x + CELL_SIZE // 2,
                                p_r * CELL_SIZE + CELL_SIZE // 2), 3)

    agent_draw_x = agent.current_pos.x - camera_x - thief_img.get_width() // 2
    agent_draw_y = agent.current_pos.y - thief_img.get_height() // 2
    screen.blit(thief_img, (agent_draw_x, agent_draw_y))

    if not can_place_block:
        remaining_cooldown = block_cooldown - time_since_last_block
        draw_text(screen, f"Block Cooldown: {remaining_cooldown:.1f}s", (10, HEIGHT - 40), 24, RED)
    else:
        draw_text(screen, "Ready to place blocks (LMB: Add, RMB: Remove)", (10, HEIGHT - 40), 24, GREEN)
    draw_text(screen, f"Speed: {agent.speed / agent.base_speed:.2f}x", (WIDTH - 150, 10), 24, BLACK)

    if game_over:
        s = pygame.Surface((WIDTH,HEIGHT), pygame.SRCALPHA)
        s.fill((50,50,50,180))
        screen.blit(s, (0,0))
        draw_text(screen, "Path Blocked or Goal Reached!", (WIDTH // 2 - 200, HEIGHT // 2 - 40), 40, RED)
        draw_text(screen, "Press 'R' to Restart or 'T' for a New Maze", (WIDTH // 2 - 250, HEIGHT // 2 + 10), 30, WHITE)

    pygame.display.flip()

pygame.quit()
sys.exit()